In [ ]:
!pip install transformers

# 2

Посмотрим, как модель справляется с выбором одной из близких о значению лексем в зависимости от контекста. Сначала посмотрим на модель с пропусками.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
model = AutoModelForMaskedLM.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased").to("cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/310 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/486k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


In [ ]:
import torch

def print_probable_words(sentence, tokenizer, model, k=10):
    """
    если бы масок было несколько, то нужно было бы находить все позиции масок,
    а потом смотреть на вероятности в каждой из этих позиций
    """
    if sentence.count("[MASK]") != 1:
        raise ValueError("Маска должна быть ровно одна.")
    tokenization = tokenizer(sentence)["input_ids"]
    index = tokenization.index(tokenizer.mask_token_id)
    # tensor: 1 * L
    tensor = torch.LongTensor([tokenization]).to("cuda")
    with torch.no_grad():
        # output = model(tensor)
        # print(output)
        # logits: L * K
        output = model(tensor)
        # print(type(output), output.keys())
        logits = output["logits"]
        # print("logits shape ", logits.shape)
        logits = logits[0]
    probs = torch.softmax(logits, dim=-1)
    # top_probs: k
    top_probs, top_indexes = torch.topk(probs[index], dim=-1, k=k)
    for token, prob in zip (tokenizer.convert_ids_to_tokens(top_indexes), top_probs):
      print(f"{token}:{prob:.3f}")

In [ ]:
sentence = "Hola, [MASK] amigo!"
print_probable_words(sentence, tokenizer, model, k=10)

viejo:0.577
mi:0.174
nuevo:0.046
¡:0.034
querido:0.027
lindo:0.011
gran:0.009
buen:0.008
tu:0.006
":0.005


In [ ]:
sentence = "Hola, [MASK] amigos!"
print_probable_words(sentence, tokenizer, model, k=10)

mis:0.521
¡:0.125
viejos:0.080
queridos:0.075
nuevos:0.063
buenos:0.053
,:0.010
!:0.006
":0.006
¿:0.005


Простые маски с разным числом местоимения модель заполняет легко.

In [ ]:
sentence = "Hace mucho [MASK]."
print_probable_words(sentence, tokenizer, model, k=10)

tiempo:0.801
[UNK]:0.055
.:0.029
que:0.011
c:0.006
-:0.005
yo:0.004
":0.004
a:0.003
d:0.002


Некоторые устойчивые выражения тоже распознает.

In [ ]:
sentence = "Tiene mucho [MASK]."
print_probable_words(sentence, tokenizer, model, k=10)

trabajo:0.080
gusto:0.067
cuidado:0.045
[UNK]:0.040
tiempo:0.036
talento:0.029
ojo:0.022
futuro:0.021
color:0.013
.:0.013


И здесь тоже.

In [ ]:
sentence = "El gato está [MASK] la mesa."
print_probable_words(sentence, tokenizer, model, k=10)

##a:0.122
de:0.105
sobre:0.103
##o:0.100
en:0.094
a:0.065
##ba:0.043
##jo:0.035
##ia:0.016
por:0.011


С предлогами начинаются некоторые трудности. Но все еще не критичные.

In [ ]:
sentence = "El problema está [MASK] la mesa."
print_probable_words(sentence, tokenizer, model, k=10)

en:0.258
es:0.144
con:0.119
de:0.105
sobre:0.068
[UNK]:0.012
a:0.011
arreglar:0.011
,:0.011
esta:0.010


Здесь мы видим, что модель с пропусками не может отличить обычного предложения от предложения с идиомой.

In [ ]:
sentence = "Si [MASK] tiempo, voy contigo."
print_probable_words(sentence, tokenizer, model, k=10)

tengo:0.364
hay:0.238
tienes:0.163
es:0.092
tiene:0.034
tenemos:0.021
me:0.013
ganas:0.008
necesito:0.006
el:0.005


Здесь проблемы нет.

In [ ]:
sentence = "Si [MASK] tiempo, iría contigo."
print_probable_words(sentence, tokenizer, model, k=10)

tienes:0.403
tengo:0.149
tiene:0.081
hay:0.057
necesitas:0.052
el:0.044
es:0.030
quieres:0.028
tenemos:0.018
hace:0.016


А здесь мы ее уже видим. Выбрано неправильное время для условного предложения.

In [ ]:
sentence = "Ella [MASK] frío."
print_probable_words(sentence, tokenizer, model, k=10)

[UNK]:0.169
es:0.086
esta:0.060
.:0.055
tiene:0.040
fue:0.038
,:0.028
estaba:0.028
no:0.025
se:0.024


Здесь все относительно хорошо.

In [ ]:
sentence = "Ella [MASK] frío en las negociaciones."
print_probable_words(sentence, tokenizer, model, k=10)

particip:0.737
no:0.029
tambien:0.025
apareci:0.016
estuvo:0.015
colabor:0.011
participa:0.010
participar:0.009
actu:0.008
fue:0.007


Здесь же мы нре увидели возможной идиомы с глаголом poner.

In [ ]:
sentence = "Él puso [MASK] la mesa un libro."
print_probable_words(sentence, tokenizer, model, k=10)

sobre:0.850
en:0.073
a:0.048
de:0.010
bajo:0.007
para:0.002
encima:0.002
con:0.001
por:0.001
entre:0.001


In [ ]:
sentence = "Él puso [MASK] la mesa el tema de la reunión."
print_probable_words(sentence, tokenizer, model, k=10)

sobre:0.955
a:0.041
en:0.003
bajo:0.001
ante:0.000
de:0.000
por:0.000
entre:0.000
con:0.000
como:0.000


Опять не распознала идиому (poner en la mesa).

# 3

Теперь посмотрим на левостороннюю модель.

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish")
model = AutoModelForCausalLM.from_pretrained("DeepESP/gpt2-spanish").to("cuda")

tokenizer_config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/914 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/840k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/499k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/261M [00:00<?, ?B/s]

In [ ]:
import torch
import numpy as np
from torch import LongTensor

def compare(texts):
  tokenizer.pad_token = tokenizer.eos_token
  batch = tokenizer(texts, return_tensors="pt", padding=True).to("cuda")
  # добавляем индекс начала строки (склейка массивов по первой координате)
  batch["input_ids"] = torch.cat([
    torch.full_like(batch["input_ids"][:,:1], tokenizer.bos_token_id),
    batch["input_ids"]
  ], dim=1)
  with torch.no_grad():
    logits = model(batch["input_ids"])["logits"]
  probs = torch.softmax(logits, dim=-1).cpu().numpy()
  tokenizer.pad_token = tokenizer.eos_token
  for i, text in enumerate(texts):
    sum_log_prob = 0.0
    print(text)
    text_token_ids = batch["input_ids"][i,1:]
    text_tokens = [x.strip("ĠĊ") for x in tokenizer.convert_ids_to_tokens(text_token_ids)]
    for j, (index, token) in enumerate(zip(text_token_ids, text_tokens)):
        print(f"{token}:{probs[i,j,index]:.3f}", end=" ")
        if index != tokenizer.eos_token_id:
            sum_log_prob += -np.log(probs[i,j,index])
    print("")
    print(f"Minus log prob: {sum_log_prob:.2f}")
    print("")

In [ ]:
texts = [
     "Hola, mi amigo!",
     "Hola, mi amigos!"
]
compare(texts)

Hola, mi amigo!
Hola:0.000 ,:0.569 mi:0.013 amigo:0.015 !:0.002 
Minus log prob: 26.25

Hola, mi amigos!
Hola:0.000 ,:0.569 mi:0.013 amigos:0.000 !:0.004 
Minus log prob: 31.78



Здесь модель справилась.

In [ ]:
texts = [
     "Hace mucho calor.",
     "Tiene mucho calor."
]
compare(texts)

Hace mucho calor.
Hace:0.000 mucho:0.054 calor:0.022 .:0.215 
Minus log prob: 17.55

Tiene mucho calor.
Tiene:0.000 mucho:0.004 calor:0.006 .:0.214 
Minus log prob: 21.91



И здесь тоже.

In [ ]:
texts = [
     "El problema está sobre la mesa.",
     "El problema está en la mesa."
]
compare(texts)

El problema está sobre la mesa.
El:0.006 problema:0.003 estÃ¡:0.013 sobre:0.002 la:0.153 mesa:0.033 .:0.304 
Minus log prob: 27.92

El problema está en la mesa.
El:0.006 problema:0.003 estÃ¡:0.013 en:0.543 la:0.089 mesa:0.000 .:0.210 
Minus log prob: 27.55



В каждом предложении мы видим, что левостороння модель лучше справляется с предложениями, чем модель с пропусками. Она, хоть и не всегда очевидно отдает предпочтение правильному/ожидаемому варианту и хорошо распознает идиомы.

In [ ]:
texts = [
     "Si tengo tiempo, voy contigo.",
     "Si tuviera tiempo, voy contigo."
]
compare(texts)

Si tengo tiempo, voy contigo.
Si:0.000 tengo:0.001 tiempo:0.018 ,:0.224 voy:0.010 contigo:0.008 .:0.459 
Minus log prob: 31.03

Si tuviera tiempo, voy contigo.
Si:0.000 tuviera:0.004 tiempo:0.013 ,:0.187 voy:0.000 contigo:0.002 .:0.469 
Minus log prob: 36.27



Со временем в условном предложении модель тоже справилась.

In [ ]:
texts = [
     "Ella tiene frío en las negociaciones.",
     "Ella puso frío en las negociaciones."
]
compare(texts)

Ella tiene frío en las negociaciones.
Ella:0.000 tiene:0.002 frÃŃo:0.001 en:0.101 las:0.054 negociaciones:0.000 .:0.268 
Minus log prob: 38.76

Ella puso frío en las negociaciones.
Ella:0.000 puso:0.003 frÃŃo:0.000 en:0.082 las:0.029 negociaciones:0.000 .:0.294 
Minus log prob: 41.31



Здесь не совсем то, но кто не ошибается.

Итак, можно сказать, что в нашем случае левосторонняя модель показала себя лучше, чем модель с пропусками.